In [32]:
import sys

import statsmodels.tsa.stattools as ts
import statsmodels.tsa.stattools as ts

#Financial data will be importing via Yahoo finance
import yfinance as yf
import scipy

import numpy as np

import datetime
from datetime import timedelta

In [39]:
#!{sys.executable} -m pip install datetime

In [71]:
def cointegrated_pairs(data):
    n =  (len(data.columns) -1)
    stocks = data.iloc[:,1:].columns.values #pulls all stock names 
    data = data.to_numpy()
    pvalue_matrix = np.zeros((n,n),dtype = float)
    pairs = np.empty((n*n,2),dtype=object)
    m = 0
    #loop through all possilbe combintions of stocks ie 3!=6
    for i in range(0,n):
        for j in range(0,n):
            S1 = data[:,i+1]
            print(S1)
            S2 = data[:,j+1]
            print(S2)
            result = ts.coint(S1,S2)
            pvalue_matrix[i,j] = result[1]
            #storing all paris in matrix form
            if (result[1] < 0.05) & (i!=j):
                pairs[m,0] = stocks[i]
                pairs[m,1] = stocks[j] 
                m = m + 1
    return pvalue_matrix,  pairs

In [65]:
#Example 2:
#SPY: SPDR S&P 500 ETF Trust
#DIA: SPDR Dow Jones Industrial Average ETF Trust

#Example 1:
#AAPL: Apple
#MSFT: Microsoft

today = datetime.date.today()
sp_list1 = ['SPY','DIA']
offset = max(1, (today.weekday() + 6) % 7 - 3)
timed = timedelta(offset)
today_business = today - timed
print("d1 =", today_business)
today = today_business.strftime("%Y-%m-%d")
start = '2010-01-01' 
end = today
print('S&P500 Stock download')
spy = yf.download(sp_list, start, end)
#print(sp_list[0])
#print(spy)

today = datetime.date.today()
sp_list2 = ['AAPL','MSFT']
offset = max(1, (today.weekday() + 6) % 7 - 3)
timed = timedelta(offset)
today_business = today - timed
print("d1 =", today_business)
today = today_business.strftime("%Y-%m-%d")
start = '2010-01-01' 
end = today
print('S&P500 Stock download')
aapl = yf.download(sp_list2, start, end)

d1 = 2022-05-13
S&P500 Stock download
[*********************100%***********************]  2 of 2 completed
d1 = 2022-05-13
S&P500 Stock download
[*********************100%***********************]  2 of 2 completed


In [70]:
#cointegrated_pairs(spy)

#scipy.stats.pearsonr(S1,S2)

print(S1)

print(S2)

#example = [[1, 2, 3],[4, 5, 6],[7, 8, 9]];

#print(example)

#print(example[2:])

#print(S1[:,5])

#scipy.stats.pearsonr(spy,spy)

[[2.76425354e+02 3.13744507e+02 2.88619995e+02 ... 3.23540009e+02
  3.86550000e+06 5.91512000e+07]
 [2.74222473e+02 3.11368744e+02 2.86320007e+02 ... 3.21160004e+02
  3.29670000e+06 7.77097000e+07]
 [2.74854706e+02 3.12556641e+02 2.86980011e+02 ... 3.20489990e+02
  1.64510000e+06 5.56539000e+07]
 ...
 [3.29070007e+02 4.11339996e+02 3.29070007e+02 ... 4.11100006e+02
  1.06433000e+07 1.51671300e+08]
 [3.22600006e+02 3.98170013e+02 3.22600006e+02 ... 4.05100006e+02
  7.81720000e+06 1.55586100e+08]
 [3.21829987e+02 3.99089996e+02 3.21829987e+02 ... 4.04489990e+02
  7.71650000e+06 1.32497200e+08]]


NameError: name 'S2' is not defined

In [44]:
result = ts.coint(S1,S2)
result[1]

NameError: name 'S1' is not defined

In [42]:
def PnL(signals):
    signals_np = np.array(signals) # converts our pandas df into numpy array
    nu_shift = np.array(signals['nu'].shift(periods=1)) #lag timeseries by 1 trading day
    vol_shift = np.array(signals['volatility'].shift(periods=1)) #lag timeseries by 1 trading day
    negvol_shift = np.array(signals['negvolatility'].shift(periods=1)) #lag timeseries by 1 trading day
    vec_sig = np.zeros(len(signals))
    for i in range(len(signals)-1):
        #innovation > volatilty & innovation+1 < volatilty+1
        if (signals_np[i,0] > signals_np[i,1]): # t1 & t0 SHORT asset when current t changes to nu > volatlity
            vec_sig[i] = -1
        elif(signals_np[i,0] < signals_np[i,2]): #LONG assets when current t changes to nu < negvol
            vec_sig[i] = 1
        else:
            vec_sig[i] = 0
    # at the current day we enter our position
    vec_sig = pd.DataFrame({'vec_sig':vec_sig})
    return vec_sig

In [43]:
def backtesting(vec_sig,S1,S2):
    num_shares_y = vec_sig['vec_sig'].iloc[1:] * 1000
    num_shares_x = vec_sig['vec_sig'].iloc[1:] * -1000
    #forward fill x & y postion 
    PLX = num_shares_x.values * S1.pct_change().iloc[1:].values # difference of column X 
    PLY = (num_shares_y.values)  * S2.pct_change().iloc[1:].values #difference of column y 
    profit_loss = PLX + PLY
buy_hold_x = 1000
    buy_hold_y = 1000
    buy_hold_x = buy_hold_x *  S1.pct_change().iloc[1:] # 
    buy_hold_y = buy_hold_y *  S2.pct_change().iloc[1:]
    buy_hold   = buy_hold_x + buy_hold_y    
    buy_hold = sum(buy_hold)
    profit_loss = sum(profit_loss)
return profit_loss , buy_hold

IndentationError: unexpected indent (1288110750.py, line 9)